## Классификация распределения Pareto и распределения Exponential

Для получения численных характеристик будем использовать дистанционный граф, так как в ходе прошлой части выяснили, что численные характеристики именно дистанционного графа лучше характеризуют распределение

На данном этапе будем использовать все численные характеристики дистанционного графа:
- Хроматическое число
- Кликовое число
- Размер максимального независимого множества
- Число доминирования
- Размер минимального кликового покрытия

Начнем с того, что соберем датасет с большим количеством симуляций построения нашего случайного графа, на котором мы будем учить нашу модель классифицировать распределения

In [1]:
import sys
sys.path.append('../../')

import numpy as np
import pandas as pd
from src.graph_builders import build_distance_graph
from src.features import compute_feature
from src.simulation import simulate_sample
from tqdm.auto import tqdm

Запустим процесс симуляции распределения и построения графа с подсчетом численных характеристик много раз и сохраним

In [3]:
# Параметры эксперимента
D = 1.5  # Фиксированный порог расстояния
SAMPLE_SIZES = [25, 100, 500]  # Размеры выборок
N_SAMPLES_PER_CLASS = {
    25: 2500,
    100: 2500,
    500: 100
}

FEATURES = [
    "chromatic_number",  # Хроматическое число
    "clique_number",     # Кликовое число (размер максимальной клики)
    "max_independent_set",  # Размер максимального независимого множества
    "domination_number",    # Число доминирования
    "clique_cover_number"   # Размер минимального кликового покрытия
]

# Параметры распределений
alpha0 = 3.0                   # для распределения Парето
lambda0 = 2.0 / np.sqrt(3.0)   # для экспоненциального распределения

PARETO_PARAMS     = {"alpha": alpha0}
EXPONENTIAL_PARAMS = {"lam": lambda0}

def compute_graph_features(sample: np.ndarray) -> list:
    G = build_distance_graph(sample, D)
    return [compute_feature(G, feature) for feature in FEATURES]



for n in tqdm([100]):
    print(f"Генерация датасета для n = {n}...")
    
    features_data = np.zeros((N_SAMPLES_PER_CLASS[n] * 2, len(FEATURES)))
    labels = np.zeros(N_SAMPLES_PER_CLASS[n] * 2)
    
    # pareto (класс 0)
    for i in tqdm(range(N_SAMPLES_PER_CLASS[n])):
        sample = simulate_sample(n, "pareto", PARETO_PARAMS)
        features_data[i] = compute_graph_features(sample)
    
    # exponential (класс 1)
    for i in tqdm(range(N_SAMPLES_PER_CLASS[n], N_SAMPLES_PER_CLASS[n] * 2)):
        sample = simulate_sample(n, "exponential", EXPONENTIAL_PARAMS)
        features_data[i] = compute_graph_features(sample)
        labels[i] = 1
    
    
    df = pd.DataFrame(features_data, columns=FEATURES)
    df['target'] = labels
    
    filename = f"generated_data/distance_graph_features_n{n}.csv"
    df.to_csv(filename, index=False)
    print(f"Сохранен датасет: {filename} (размер: {df.shape})")

  0%|          | 0/1 [00:00<?, ?it/s]

Генерация датасета для n = 100...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Сохранен датасет: generated_data/distance_graph_features_n100.csv (размер: (5000, 6))


Теперь рассмотрим 3 различных классификатора и обучим их. Использовать будем следующие классификаторы, которые позволяют интерпретировать значимость признаков:
- Логистическая регрессия
- Случайный лес
- Градиентный бустинг

In [4]:
# Импортируем все нужное, чтобы учить классификаторы

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)

import time
import os

In [7]:
!tree

.
├── confusion_matrices
├── feature_importances
├── generated_data
│   ├── distance_graph_features_n100.csv
│   ├── distance_graph_features_n25.csv
│   └── distance_graph_features_n500.csv
├── pareto_exp_classification.ipynb
└── pareto_exp_exploration.ipynb

4 directories, 5 files


In [8]:
def load_and_prepare_data(n_size):
    filename = f"generated_data/distance_graph_features_n{n_size}.csv"
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Файл {filename} не найден! Сначала сгенерируйте данные.")
    
    df = pd.read_csv(filename)
    
    X = df.drop('target', axis=1)
    y = df['target']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

In [9]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, n_size):
    results = []
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "CatBoost": CatBoostClassifier(iterations=500, learning_rate=0.05, 
                                      depth=6, verbose=False, random_state=42)
    }
    
    for i, (model_name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        # Расчет метрик
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Сохранение результатов
        results.append({
            'Model': model_name,
            'Size': n_size,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
        })
    
    return pd.DataFrame(results)

In [10]:
def analyze_feature_importance(models, feature_names, n_size):
    plt.figure(figsize=(12, 8))
    
    for i, (model_name, model) in enumerate(models.items()):
        if model_name == "Logistic Regression":
            importances = np.abs(model.coef_[0])
        elif model_name == "Random Forest":
            importances = model.feature_importances_
        elif model_name == "CatBoost":
            importances = model.get_feature_importance()
        else:
            continue
        
        importances = 100.0 * (importances / importances.max())
        sorted_idx = np.argsort(importances)
        
        plt.subplot(3, 1, i+1)
        plt.barh(range(len(importances)), importances[sorted_idx], align='center')
        plt.yticks(range(len(importances)), [feature_names[i] for i in sorted_idx])
        plt.xlabel('Важность признака (%)')
        plt.title(f'{model_name} - Важность признаков (n={n_size})')
    
    plt.tight_layout()
    plt.savefig(f'feature_importances/feature_importance_n{n_size}.png', dpi=300)
    plt.close()

In [11]:
# Размеры выборок для анализа
sample_sizes = [25, 100, 500]

all_results = []

for n_size in sample_sizes:
    print(f"\n{'='*50}")
    print(f"Анализ для размера выборки n = {n_size}")
    print(f"{'='*50}")
    
    X_train, X_test, y_train, y_test, scaler = load_and_prepare_data(n_size)
    feature_names = ["chromatic_number", "clique_number", 
                    "max_independent_set", "domination_number", 
                    "clique_cover_number"]
    
    size_results = train_and_evaluate_models(X_train, X_test, y_train, y_test, n_size)
    all_results.append(size_results)
    
    print(f"\nРезультаты для n={n_size}:")
    print(size_results)
    
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train),
        "CatBoost": CatBoostClassifier(iterations=500, learning_rate=0.05, 
                                     depth=6, verbose=False, random_state=42).fit(X_train, y_train)
    }
    analyze_feature_importance(models, feature_names, n_size)

final_results = pd.concat(all_results)
print("\nИтоговые результаты по всем моделям и размерам выборок:")
print(final_results)


Анализ для размера выборки n = 25

Результаты для n=25:
                 Model  Size  Accuracy  Precision  Recall        F1
0  Logistic Regression    25     0.814   0.829832   0.790  0.809426
1        Random Forest    25     0.814   0.835470   0.782  0.807851
2             CatBoost    25     0.815   0.835821   0.784  0.809082

Анализ для размера выборки n = 100

Результаты для n=100:
                 Model  Size  Accuracy  Precision  Recall        F1
0  Logistic Regression   100     0.977   0.979879   0.974  0.976931
1        Random Forest   100     0.977   0.979879   0.974  0.976931
2             CatBoost   100     0.977   0.979879   0.974  0.976931

Анализ для размера выборки n = 500

Результаты для n=500:
                 Model  Size  Accuracy  Precision  Recall   F1
0  Logistic Regression   500       1.0        1.0     1.0  1.0
1        Random Forest   500       1.0        1.0     1.0  1.0
2             CatBoost   500       1.0        1.0     1.0  1.0

Итоговые результаты по всем 

Как мы видим, все три модели показали хорошие результаты

Сделаем выводы о вероятности ошибки первого рода и мощности построенных классификаторов, если рассматривать их как статистический критерий

In [12]:
def evaluate_statistical_metrics(X_test, y_test, models, n_size):
    results = []
    
    # Создаем фигуру для матриц ошибок
    fig, axes = plt.subplots(1, len(models), figsize=(5*len(models), 4))
    if len(models) == 1:
        axes = [axes]
    
    fig.suptitle(f'Матрицы ошибок (n={n_size})', fontsize=16)
    
    for i, (model_name, model) in enumerate(models.items()):
        y_pred = model.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        
        # Ошибка первого рода = P(отвергнуть H0|H0 верна) = FP / (FP + TN)
        type1_error = fp / (fp + tn)
        
        # Мощность = P(отвергнуть H0|H1 верна) = TP / (TP + FN)
        power = tp / (tp + fn)
        
        results.append({
            'Model': model_name,
            'Size': n_size,
            'Type I Error': type1_error,
            'Power': power
        })
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                   xticklabels=['Pareto (H₀)', 'Exponential (H₁)'], 
                   yticklabels=['Pareto (H₀)', 'Exponential (H₁)'])
        axes[i].set_title(f'{model_name}\nType I: {type1_error:.4f}, Power: {power:.4f}')
        axes[i].set_xlabel('Predicted')
        axes[i].set_ylabel('True')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f'confusion_matrices/statistical_metrics_n{n_size}.png', dpi=300)
    plt.close()
    
    return pd.DataFrame(results)

In [13]:
statistical_results = []

for n_size in sample_sizes:
    print(f"\n{'='*50}")
    print(f"Статистический анализ для размера выборки n = {n_size}")
    print(f"{'='*50}")
    
    X_train, X_test, y_train, y_test, scaler = load_and_prepare_data(n_size)
    
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train),
        "CatBoost": CatBoostClassifier(iterations=500, learning_rate=0.05, 
                                     depth=6, verbose=False, random_state=42).fit(X_train, y_train)
    }
    
    size_stats = evaluate_statistical_metrics(X_test, y_test, models, n_size)
    statistical_results.append(size_stats)
    
    print(f"\nСтатистические метрики для n={n_size}:")
    print(size_stats[['Model', 'Type I Error', 'Power']])

final_stat_results = pd.concat(statistical_results)
print("\nИтоговые статистические метрики по всем моделям и размерам выборок:")
print(final_stat_results[['Model', 'Size', 'Type I Error', 'Power']])


Статистический анализ для размера выборки n = 25

Статистические метрики для n=25:
                 Model  Type I Error  Power
0  Logistic Regression         0.162  0.790
1        Random Forest         0.154  0.782
2             CatBoost         0.154  0.784

Статистический анализ для размера выборки n = 100

Статистические метрики для n=100:
                 Model  Type I Error  Power
0  Logistic Regression          0.02  0.974
1        Random Forest          0.02  0.974
2             CatBoost          0.02  0.974

Статистический анализ для размера выборки n = 500

Статистические метрики для n=500:
                 Model  Type I Error  Power
0  Logistic Regression           0.0    1.0
1        Random Forest           0.0    1.0
2             CatBoost           0.0    1.0

Итоговые статистические метрики по всем моделям и размерам выборок:
                 Model  Size  Type I Error  Power
0  Logistic Regression    25         0.162  0.790
1        Random Forest    25         0.154  0.7